In [ ]:
import itertools
import subprocess
from collections import defaultdict
import math
import time

In [ ]:
var_dict = {}
rev_dict = {}
last_id = 1
product_num = 0

In [ ]:
def parse_edge_string(edge_string):
    edge_string = edge_string.strip().strip("e_{").strip("}")
    i, j = map(int, edge_string.split(","))
    return (i, j)

In [ ]:
def add_var_id(x, old_id=None):
    global last_id, var_dict, rev_dict
    xlast_id = 'x' + str(last_id)
    if old_id != None:
        var_dict[x] = old_id
        return
    var_dict[x] = xlast_id
    x = x + '}'
    x = x[0] + '_{' + x[1:]
    rev_dict[xlast_id] = x
    rev_dict['-' + xlast_id] = x
    last_id += 1
    return xlast_id

In [ ]:
def srg(n, K, la, mu):
    global product_num
    clauses = []
    
    for i, j in itertools.combinations(range(n), 2):
        edge_id = add_var_id(f"e{i},{j}")
        add_var_id(f"e{j},{i}", edge_id)
    
    for i in range(n):
        for j in range(i + 1, n):
            edge = var_dict[f"e{i},{j}"]
            la_clause = f"-{la} {edge}"
            mu_clause = f"+{mu} {edge}"
            for k in range(n):
                if k == i or k == j:
                    continue
                edge_1 = var_dict[f"e{i},{k}"]
                edge_2 = var_dict[f"e{k},{j}"]
                la_clause = la_clause + f" +1 {edge} {edge_1} {edge_2}"
                product_num += 1
                mu_clause = mu_clause + f" +1 {edge_1} {edge_2} -1 {edge} {edge_1} {edge_2}"
                product_num += 1
            la_clause += f" = 0"
            mu_clause += f" = {mu}"
            clauses.append(la_clause)
            clauses.append(mu_clause)
    
#     regularity
    for i in range(n):
        clause = ""
        for j in range(n):
            if j == i:
                continue
            e = var_dict[f"e{i},{j}"]
            clause = clause + f"+1 {e} "
        clause = clause + f"= {K}"
        clauses.append(clause)
    
    return clauses

In [ ]:
def parse_clasp_output(output):
    lines = output.strip().split("\n")
    solution = []
    for line in lines:
        if line.startswith("v"):
            variables = line[2:].split()
            for var in variables:
                solution.append(var)
    return solution

In [ ]:
srg_string = "5K2L0M1"
N, K, LAMBDA, MU = map(int, srg_string.replace("K", " ").replace("L", " ").replace("M", " ").split())
c = srg(N,K, LAMBDA, MU)
file = open(f"solutions/{N}K{K}L{LAMBDA}M{MU}.txt", 'r')
output = file.read()
solution = parse_clasp_output(output)
edges_list = []
if len(solution) == 0:
    print("UNSATISFIABLE")
else:
    for var in solution:
        if var.startswith('x'):
            res = rev_dict[var]
            if res.startswith('e'):
                edges_list.append(parse_edge_string(res))

In [ ]:
def check_srg(N, K, LAMBDA, MU, edges_list):
    # Build adjacency list and count degrees
    adjacency_list = defaultdict(set)
    degrees = defaultdict(int)
    for u, v in edges_list:
        adjacency_list[u].add(v)
        adjacency_list[v].add(u)
        degrees[u] += 1
        degrees[v] += 1

    if not all(degree == K for degree in degrees.values()):
        return False

    for u, v in edges_list:
        common_neighbors = adjacency_list[u] & adjacency_list[v]
        if len(common_neighbors) != LAMBDA:
            return False
    for u in range(N):
        for v in range(u + 1, N):
            if (u, v) not in edges_list and (v, u) not in edges_list:
                common_neighbors = adjacency_list[u] & adjacency_list[v]
                if len(common_neighbors) != MU:
                    return False

    return True

In [ ]:
def graph_to_txt(n, edges):
    row_strings = []
    for i in range(n):
        row = ["0" if (i, j) not in edges and (j, i) not in edges else "1" for j in range(n)]
        row_strings.append("".join(row))
    csv_content = "\n".join(row_strings)
    with open(f"srgs_txts/srgN{N}K{K}L{LAMBDA}M{MU}.txt", "w") as file:
        file.write(csv_content)

In [ ]:
if check_srg(N, K, LAMBDA, MU, edges_list):
    print("Result is strongly regular")
    graph_to_txt(N, edges_list)
else:
    print("wrong")